In [1]:
# importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.utils import resample
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [2]:
# uploading dataset
orders = pd.read_csv(r"C:\Users\zulhu\Desktop\pandas-brazilian-ecommerce\olist_public_dataset_v3.csv")

# converting to datetime
orders['order_purchase_timestamp'] = pd.to_datetime(orders.order_purchase_timestamp)
orders['order_aproved_at'] = pd.to_datetime(orders.order_aproved_at).dt.date  
orders['order_estimated_delivery_date'] = pd.to_datetime(orders.order_estimated_delivery_date).dt.date  
orders['order_delivered_customer_date'] = pd.to_datetime(orders.order_delivered_customer_date).dt.date  

# get translations for category names
translation = pd.read_csv(r"C:\Users\zulhu\Desktop\pandas-brazilian-ecommerce\product_category_name_translation.csv")
orders = orders.merge(translation, on='product_category_name').drop('product_category_name', axis=1)

In [3]:
# creating label encoding
le_status = LabelEncoder()
le_state = LabelEncoder()
le_category = LabelEncoder()

le_status.fit(orders.order_status)
le_state.fit(orders.customer_state)
le_category.fit(orders.product_category_name_english)

# transforming label
np.save('Encoder\le_status.npy', le_status.classes_)
np.save('Encoder\le_state.npy', le_state.classes_)
np.save('Encoder\le_category.npy', le_category.classes_)

status_encoded = le_status.transform(orders.order_status)
state_encoded = le_state.transform(orders.customer_state)
category_encoded = le_category.transform(orders.product_category_name_english)

# creating one-hot encoding
status_ohe = Pipeline([('OneHotEncoder', OneHotEncoder())])
state_ohe = Pipeline([('OneHotEncoder', OneHotEncoder())])
category_ohe = Pipeline([('OneHotEncoder', OneHotEncoder())])

status_ohe.fit(status_encoded.reshape(-1,1))
state_ohe.fit(state_encoded.reshape(-1,1))
category_ohe.fit(category_encoded.reshape(-1,1))

joblib.dump(status_ohe,"Encoder\status_ohe.joblib")
joblib.dump(state_ohe,"Encoder\state_ohe.joblib")
joblib.dump(category_ohe,"Encoder\category_ohe.joblib")

C:\Users\zulhu\Miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\zulhu\Miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you 

['Encoder\\category_ohe.joblib']

In [4]:
# separating the classes
score_1 = orders[orders.review_score==1]
score_2 = orders[orders.review_score==2]
score_3 = orders[orders.review_score==3]
score_4 = orders[orders.review_score==4]
score_5 = orders[orders.review_score==5]

In [5]:
# downsampling to match smallest class
#score1_downsampled = resample(score_1, 
#                                 replace=True,    # sample without replacement
#                                 n_samples=len(score_5.index),     # to match minority class
#                                 random_state=123) # reproducible results
#
#score2_downsampled = resample(score_2, 
#                                 replace=True,    # sample without replacement
#                                 n_samples=len(score_5.index),     # to match minority class
#                                 random_state=123) # reproducible results
#
#score3_downsampled = resample(score_3, 
#                                 replace=True,    # sample without replacement
#                                 n_samples=len(score_5.index),     # to match minority class
#                                 random_state=123) # reproducible results
#
#score4_downsampled = resample(score_4, 
#                                 replace=True,    # sample without replacement
#                                 n_samples=len(score_5.index),     # to match minority class
#                                 random_state=123) # reproducible results
#


In [6]:
not_satisfied = pd.concat([score_1, score_2])
not_satisfied = pd.concat([not_satisfied, score_3])
not_satisfied = pd.concat([not_satisfied, score_4])
not_satisfied.review_score = 0
satisfied = resample(score_5, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(not_satisfied.index),     # to match minority class
                                 random_state=123) # reproducible results
satisfied.review_score = 1
# orders_downsampled.review_score.value_counts()
# len(orders_downsampled.index)
new_orders = pd.concat([not_satisfied, satisfied])
new_orders = new_orders.reset_index(drop=True)
new_orders

C:\Users\zulhu\Miniconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_state,product_name_lenght,product_description_lenght,product_photos_qty,review_score,product_category_name_english
0,delivered,109.78,20.26,1,1,2017-02-08 18:58:00,2017-02-08,2017-08-24,2017-08-14,RJ,56,1770,1,0,health_beauty
1,delivered,147.90,15.79,1,1,2017-06-14 19:28:00,2017-06-14,2017-07-07,2017-12-07,MG,63,1194,1,0,health_beauty
2,canceled,184.10,14.63,1,1,2018-07-14 17:10:00,2018-07-14,2018-03-08,NaT,MG,49,571,2,0,health_beauty
3,delivered,49.99,11.73,1,1,2017-08-21 21:58:00,2017-08-21,2017-04-09,2017-08-28,SP,56,1534,1,0,health_beauty
4,delivered,109.90,24.42,1,1,2018-06-03 13:15:00,2018-06-03,2018-02-04,2018-02-05,RJ,54,762,1,0,health_beauty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86609,delivered,25.00,11.85,1,1,2017-08-23 12:17:00,2017-08-24,2017-09-14,2017-08-29,SP,58,419,2,1,computers_accessories
86610,delivered,649.17,22.14,1,1,2016-06-10 10:12:00,2016-06-10,2016-06-12,2016-10-27,DF,44,1496,3,1,watches_gifts
86611,delivered,72.90,14.09,1,1,2018-07-29 17:25:00,2018-07-30,2018-08-13,2018-04-08,SP,35,155,1,1,bed_bath_table
86612,delivered,89.99,7.88,1,1,2017-07-20 16:09:00,2017-07-20,2017-04-08,2017-07-26,SP,59,492,3,1,health_beauty


In [7]:
# Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

# Stratified Split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=123)
for train_index, test_index in split.split(new_orders, new_orders['review_score']):
    strat_train_set = new_orders.loc[train_index]
    strat_test_set = new_orders.loc[test_index]

In [8]:
# modifying the dataset to produce workable features
from workalendar.america import Brazil
cal = Brazil()
from sklearn.base import BaseEstimator, TransformerMixin

class AttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df = X.copy()
        
        # Calculate the estimated delivery time and actual delivery time in working days. 
        # This would allow us to exclude hollidays that could influence delivery times.
        # If the order_delivered_customer_date is null, it returns 0.
        df['wd_estimated_delivery_time'] = df.apply(lambda x: cal.get_working_days_delta(x.order_aproved_at, 
                                                                                      x.order_estimated_delivery_date), axis=1)
        df['wd_actual_delivery_time'] = df.apply(lambda x: cal.get_working_days_delta(x.order_aproved_at, 
                                                                                   x.order_delivered_customer_date), axis=1)

        # Calculate the time between the actual and estimated delivery date. If negative was delivered early, if positive was delivered late.
        df['wd_delivery_time_delta'] = df.wd_actual_delivery_time - df.wd_estimated_delivery_time


        # Calculate the time between the actual and estimated delivery date. If negative was delivered early, if positive was delivered late.
        df['is_late'] = df.order_delivered_customer_date > df.order_estimated_delivery_date
        
        # Calculate the average product value.
        df['average_product_value'] = df.order_products_value / df.order_items_qty

        # Calculate the total order value
        df['total_order_value'] = df.order_products_value + df.order_freight_value
        
        # Calculate the order freight ratio.
        df['order_freight_ratio'] = df.order_freight_value / df.order_products_value
        
        # Calculate the order freight ratio.
        df['purchase_dayofweek'] = df.order_purchase_timestamp.dt.dayofweek
                       
        # With that we can remove the timestamps from the dataset
        cols2drop = ['order_purchase_timestamp', 'order_aproved_at', 'order_estimated_delivery_date', 
                     'order_delivered_customer_date']
        df.drop(cols2drop, axis=1, inplace=True)
        
        return df

In [9]:
# Executing the estimator
attr_adder = AttributesAdder()
feat_eng = attr_adder.transform(strat_train_set)
feat_eng = feat_eng.reset_index(drop=True)

In [10]:
# creating label encoding
le_late = LabelEncoder()
le_late.fit(feat_eng.is_late)
np.save('Encoder\le_late.npy', le_late.classes_)
late_encoded = le_late.transform(feat_eng.is_late)

# transforming label
feat_eng['status_encoded'] = le_status.transform(feat_eng.order_status)
feat_eng['state_encoded'] = le_state.transform(feat_eng.customer_state)
feat_eng['category_encoded'] = le_category.transform(feat_eng.product_category_name_english)
feat_eng['late_encoded'] = le_late.transform(feat_eng.is_late)

# creating one-hot encoding
late_ohe = Pipeline([('OneHotEncoder', OneHotEncoder())])
late_ohe.fit(late_encoded.reshape(-1,1))
joblib.dump(late_ohe,"Encoder\late_ohe.joblib")

# transforming one hot
x_status = status_ohe.transform(feat_eng.status_encoded.values.reshape(-1,1)).toarray()
x_state = state_ohe.transform(feat_eng.state_encoded.values.reshape(-1,1)).toarray()
x_category = category_ohe.transform(feat_eng.category_encoded.values.reshape(-1,1)).toarray()
x_late = late_ohe.transform(feat_eng.late_encoded.values.reshape(-1,1)).toarray()

C:\Users\zulhu\Miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
status_list = pd.DataFrame(list(le_status.classes_))
state_list = pd.DataFrame(list(le_state.classes_))
category_list = pd.DataFrame(list(le_category.classes_))
late_list = pd.DataFrame(list(le_late.classes_))

In [12]:
dfOneHot = pd.DataFrame(x_status, columns = ["status"+str(int(i)) for i in range(x_status.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(x_state, columns = ["state"+str(int(i)) for i in range(x_state.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(x_category, columns = ["category"+str(int(i)) for i in range(x_category.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(x_late, columns = ["late"+str(int(i)) for i in range(x_late.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

In [13]:
# remove categorical columns
cols2drop = ['status_encoded', 'state_encoded', 'category_encoded', 'late_encoded', 'product_category_name_english', 
             'customer_state', 'is_late', 'order_status']
feat_eng.drop(cols2drop, axis=1, inplace=True)

In [14]:
feat_eng.to_csv(r'feat_eng.csv', index = False)

In [15]:
# Executing the estimator
attr_adder = AttributesAdder()
feat_eng = attr_adder.transform(strat_test_set)
feat_eng = feat_eng.reset_index(drop=True)

In [16]:
# transforming label
feat_eng['status_encoded'] = le_status.transform(feat_eng.order_status)
feat_eng['state_encoded'] = le_state.transform(feat_eng.customer_state)
feat_eng['category_encoded'] = le_category.transform(feat_eng.product_category_name_english)
feat_eng['late_encoded'] = le_late.transform(feat_eng.is_late)

# transforming one hot
x_status = status_ohe.transform(feat_eng.status_encoded.values.reshape(-1,1)).toarray()
x_state = state_ohe.transform(feat_eng.state_encoded.values.reshape(-1,1)).toarray()
x_category = category_ohe.transform(feat_eng.category_encoded.values.reshape(-1,1)).toarray()
x_late = late_ohe.transform(feat_eng.late_encoded.values.reshape(-1,1)).toarray()

In [17]:
dfOneHot = pd.DataFrame(x_status, columns = ["status"+str(int(i)) for i in range(x_status.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(x_state, columns = ["state"+str(int(i)) for i in range(x_state.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(x_category, columns = ["category"+str(int(i)) for i in range(x_category.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(x_late, columns = ["late"+str(int(i)) for i in range(x_late.shape[1])])
feat_eng = pd.concat([feat_eng, dfOneHot], axis=1)

In [18]:
# remove categorical columns
cols2drop = ['status_encoded', 'state_encoded', 'category_encoded', 'late_encoded', 'product_category_name_english', 
             'customer_state', 'is_late', 'order_status']
feat_eng.drop(cols2drop, axis=1, inplace=True)

In [19]:
feat_eng.to_csv(r'feat_eng_test.csv', index = False)

In [20]:
status_list

,0
0,approved
1,canceled
2,delivered
3,invoiced
4,processing
5,shipped
6,unavailable


In [21]:
state_list

,0
0,AC
1,AL
2,AM
3,AP
4,BA
5,CE
6,DF
7,ES
8,GO
9,MA


In [22]:
category_list

,0
0,agro_industry_and_commerce
1,air_conditioning
2,art
3,arts_and_craftmanship
4,audio
...,...
66,stationery
67,tablets_printing_image
68,telephony
69,toys


In [23]:
late_list

,0
0,False
1,True
